<font size="6"> <center>Efficient Debugging with Python</center>
<font size="4"> <center>lsimek.github.io</center> </font>

# Introduction
Debugging will never be as glamorous and interesting as the 'meat' of software-related projects, but it is still an unavoidable part of the process. We have all been there &ndash; finishing our code relatively quickly before spending hours on discovering and removing needles in a haystack. There are better and worse ways to go about this, that in turn will take up less or more of your time. This notebook aims to showcase some more sophisticated and efficient ways of debugging.

# Preventative Measures
The first way to avoid hours of debugging is to avoid making many mistakes in the first place. For example, one should be very clear on what they are doing before they start writing the respective code. Be clear about what you want, which algorithms you are using and, if need be, exactly how they function. This also goes for third-party tools like additional libraries. This will take up some time in the short run, but is ultimately preferrable to causing unexpected errors before spending a much longer time fidgeting with the details. Worse yet, it may turn out that the library cannot actually do what you wish or that the algorithm you envisioned is not completely correct.

I recommend that you consult the documentation of third party tools that are new to you, especially if they are more niche. It will usually contain all you need to know, and may even warn you about some common pitfalls. This should be preferred over consulting AI chatbots as they will, in my own experience, offer solutions that are deprecated, wrong or hallucinated. They also have a tendency to get stuck in infinite loops of repeated wrong answers and wrong corrections. Even when they offer correct solutions, they may miss a far more elegant one, leaving you with one that is clunkier, less efficient or looks amateurish. Moreover, a solution to a given small problem may indeed work, but this way of solving it will still not teach you about the larger context of your problem, taking away a natural learning opportunity. AI chatbots may be useful for problems that are more basic or that are peripheral to your work, but are overall greatly overrated.

# Debugging is Already Here <a id='#already'></a>
The Python interpreter offers a lot of valuable information on its own. When it encounters a runtime error, it will print out a traceback of function calls, exception types and messages, also involving line numbers, module names and snippets of the source code. One should learn to read these 'reports' and recognize different kinds of errors as this information is often all that you need. Of course, the raising of an `IndexError` by a library of whose inner working you know little of may not be of use &ndash; here it is again advisable to read the documentation or, if need be, dig deeper. Many libraries and frameworks provide their own 'debug mode' that offers more verbose logging or diagnostic information.

There also exist easily accessible debuggers, such as Python's standard debuggers (pdb and ipdb) or those provided by IDEs. These tools are very flexible and can provide many kinds of information, but this is paid for with greater difficulty of use. Additionaly, different debuggers have distinctive commands or interfaces and may be platform dependent as is the case of debuggers provided by IDEs. Still, these tools can be very valuable and one should learn to utilize them, even if they are not the best idea in all circumstances. In Jupyter notebooks, a debugger session can be initiated with the magic `%debug`.

# `print` Lines
So-called `print` lines are somewhat infamous. They are inflexible and require manual writing of messages, insertions into code and later removal. This means that they can quickly inflate your code and its output to the point where it is difficult to read or interpret. Nevertheless, in some cases there is really no need to complicate. Python's f-strings are very helpful in this regard. Many people do not know that the f-string `f'{x=}'` is equivalent to `f'x={x}'`.

In [2]:
x, y = 4, 2
print(f'{x=}, {x/y = }')

x=4, x/y = 2.0


# `assert` Statements
The `assert` keyword will check a given condition and raise `AssertionError` in the case it is false. An additional message may be provided for debugging purposes. The exception will be raised and the message printed conditionally, making them less versatile but more concise, while also having the ability to stop execution. Also, note that passing the optimizing `-O` flag to the interpreter will make it ignore assert statements.

In [3]:
import numpy as np

# calculate compounded rate of return from percentages
def compounded_ror(array):
    return (1+array).prod(axis=array.ndim-1)-1

# check that inputs are valid before calling function
array = np.array([0.04, 0.045, 1.1])
assert np.all((array >= 0) & (array <= 1)), f'Invalid inputs to compounded_ror: {array[(array < 0) | (array > 1)]} (must be percentages between 0 and 1)'
compounded_ror(array)

AssertionError: Invalid inputs to compounded_ror: [1.1] (must be percentages between 0 and 1)

# Logging
Logging can achieved done with the `logging` module of Python's standard library ([docs](https://docs.python.org/3/library/logging.html)). Logging is a welcome part of most projects and is not strictly tied to purposed of debugging. It is very flexible for the following reasons:
- we can utilize many loggers (`logging.Logger` objects), each with its own 'rules'.
- there are more and less verbose levels, starting with `DEBUG` followed by `INFO`, `WARNING`, `ERROR` and `CRITICAL`. This is a setting that can be easily changed depending on our needs, e.g. in a configuration file or via a command line argument. Additional (custom) levels may be introduced.
- similarly, each logger may have a handler (`logging.Handler`, or rather a subclass such as `logging.StreamHandler` or `logging.FileHandler`) controlling the output (e.g. to the console, to a file, to a network socket, or conditional output depending on size). Handlers also have formatters and filters (discussed in a moment). Multiple handlers may be used for the same logger, making each do its own work. A handler may easily be replaced with a different one depending on our current needs.
- a formatter will choose the format and contents of a message. The logging module's LogRecords by themselves contain a lot of useful information like function names, line numbers, timestamps, etc. The format itself can vary, from separators to organizations into lines or text coloring.
- a filter will filter out records to be logged based on some criteria. We already mentioned the logging levels.
- additional subclasses of the base classes `logging.Handler`, `logging.Formatter` and `logging.Filter` may be introduced in simple ways.

Many loggers can be organized in a hierarchy. The command `logging.log(...)` is referring to the root logger, while `logging.getLogger()` creates a child of the root logger. More can be found in the documentation.

In [4]:
import logging
import sys

from importlib import reload
reload(logging)
"""
importrant for jupyter: otherwiser we would add a handler
everytime this cell is executed, resulting in
repeated outputs
"""

logger = logging.getLogger('debug-logger')  # creating and naming a logger
logger.setLevel(logging.DEBUG)  # show all logs

stdout_handler = logging.StreamHandler(sys.stdout)  # log to stdout

class ColoringFormatter(logging.Formatter):  # our custom formatter
    colors = {
        'red': '\033[91m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'blue': '\033[94m',
        'reset': '\033[0m',
    }

    def format(self, record):  # overriding base class' method
        timestamp = self.formatTime(record, '%H:%M:%S')
        lineno = record.lineno
        funcname = record.funcName
        message = record.getMessage()
        level = record.levelname

        return f"{self.colors['red']}{timestamp}{self.colors['reset']}-"\
        f"{self.colors['red']}{level}{self.colors['reset']}-"\
        f"{self.colors['green']}at line {lineno}{self.colors['reset']}-"\
        f"{self.colors['yellow']}in func {funcname}{self.colors['reset']}-"\
        f"{self.colors['blue']}{message}{self.colors['reset']}"
        # alternatively, we could have chosen the color
        # based on the logging level

# simpler
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(funcName)s - %(message)s')

stdout_handler.setFormatter(ColoringFormatter())  # adding formatter to handler
logger.addHandler(stdout_handler)  # adding handler to logger

In [5]:
def check_condition(array):
    logger.debug('Entering')
    return np.all((array >= 0) & (array <=1), axis=array.ndim-1)

def compound_ror(array):
    logger.debug(f'Entering with {array=}')
    if not check_condition(array):
        raise ValueError('The inputs must be percentages between 0 and 1')

    return np.prod(array+1, axis=array.ndim-1)-1

arr1 = np.array([0.04, 0.044, 0.033])
arr2 = np.array([0.6, 0.88, 1.1])

for arr in [arr1, arr2]:
    try:
        print(compound_ror(arr))
    except ValueError:
        logger.error('Caught ValueError')

23:28:58-DEBUG-at line 6-in func compound_ror-Entering with array=array([0.04 , 0.044, 0.033])
23:28:58-DEBUG-at line 2-in func check_condition-Entering
0.12159008000000004
23:28:58-DEBUG-at line 6-in func compound_ror-Entering with array=array([0.6 , 0.88, 1.1 ])
23:28:58-DEBUG-at line 2-in func check_condition-Entering
23:28:58-ERROR-at line 19-in func <module>-Caught ValueError


Note that this is an example &ndash; logging should always be concise and informative, as opposed to cluttering the output.

# Wrappers
The `inspect` module ([docs](https://docs.python.org/3/library/inspect.html)) enables us to capture varied information about live objects, expanding on functions like `help` or `vars` and `dir`. These objects range from modules to functions and the information includes things like names and line numbers (of definition) but also source code. Again, this module's purpose is not restricted to debugging, though it can be useful there as well.

Debugging can also be achieved with dedicated debugging wrappers. Wrappers are powerful because:
- any debugging logic can be encapsulated and used across various functions and even modules
- wrappers can be very simply added, removed or replaced by use of decorators

A simple wrapper could just log calls to functions and the respective arguments, while a more complex one could execute the function line by line, providing information about all changes. Keeping such more powerful wrappers flexible enough to be function-agnostic will also require more effort. Such a wrapper should take care of initializing local variables, variations in possible syntax, and be careful with in-place operations (executing the code of such a function twice may produce unwanted effects).

The example that follows uses `inspect` to fetch the source code of a function, enabling us to write a wrapper that will, after each line, print out the values of local variables. This wrapper is still simple as it cannot handle indentation (`exec` has to execute if-blocks, for-blocks and similar as a single command). It also does not handle (possibly) positional arguments with default values. This can still be improved with more complex parsing. The standard module `ast` can be helpful for that purpose.

In [6]:
import inspect

class DebugWrapper:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        func = self.func
        logger.debug(f'Entering function {func.__name__} with {args=} and {kwargs=}')
        
        iargs = iter(args)
        ikwargs = iter(kwargs)
    
        source_lines, _ = inspect.getsourcelines(func)
    
        for i, line in enumerate(source_lines):
            if line.startswith('@'):  # skip decorators
                continue
    
            if line.startswith('def'):  # parsing the signature
                line = line[
                    line.find('(')+1:
                    line.find(')')
                ]
    
                elements = line.split(',')
                for element in elements:
                    if '=' in element:  # has default value
                        exec(element.strip(), func.__globals__, locals())
                    else:
                        exec(f'{element.strip()}={next(iargs)}', func.__globals__, locals())
    
                for kwarg, value in kwargs.items():
                    exec(f'{kwarg}=value', func.__globals__, locals())
                
                del iargs
                del ikwargs
                del elements
                continue

            line = line.strip()
            if 'return' not in line:
                exec(line, func.__globals__, locals())
    
                local_vars = {}
                for key, value in locals().items():
                    if (not key in ['args', 'kwargs', 'source_lines', 'remove_indent', 'def_flag', 'local_vars', 'line', 'value', 'key', 'element', '_', 'func', 'kwarg', 'self', 'i']):
                        local_vars[key] = value
                logger.debug(f'Executing line {i}...{local_vars=}')
    
            else:
                expr = line.split('return')[1]
                value = eval(expr)
                logger.debug(f'Exiting function {func.__name__} with value {expr}={value}')
                return value
                
        logger.debug(f'Exiting function {func.__name__}')
        return None

For now, let us test the wrapper on a simple function.

In [7]:
@DebugWrapper
def simple_function(a, b, c=5):
    d = a+b
    e = d*c
    f = e ** 2
    return f

simple_function(3, 4)
simple_function(1, 3, c=9)

23:28:59-DEBUG-at line 9-in func __call__-Entering function simple_function with args=(3, 4) and kwargs={}
23:28:59-DEBUG-at line 49-in func __call__-Executing line 2...local_vars={'a': 3, 'b': 4, 'c': 5, 'd': 7}
23:28:59-DEBUG-at line 49-in func __call__-Executing line 3...local_vars={'a': 3, 'b': 4, 'c': 5, 'd': 7, 'e': 35}
23:28:59-DEBUG-at line 49-in func __call__-Executing line 4...local_vars={'a': 3, 'b': 4, 'c': 5, 'd': 7, 'e': 35, 'f': 1225}
23:28:59-DEBUG-at line 54-in func __call__-Exiting function simple_function with value  f=1225
23:28:59-DEBUG-at line 9-in func __call__-Entering function simple_function with args=(1, 3) and kwargs={'c': 9}
23:28:59-DEBUG-at line 49-in func __call__-Executing line 2...local_vars={'a': 1, 'b': 3, 'c': 9, 'd': 4}
23:28:59-DEBUG-at line 49-in func __call__-Executing line 3...local_vars={'a': 1, 'b': 3, 'c': 9, 'd': 4, 'e': 36}
23:28:59-DEBUG-at line 49-in func __call__-Executing line 4...local_vars={'a': 1, 'b': 3, 'c': 9, 'd': 4, 'e': 36, 'f

1296

# Metaclasses
Metaclasses are classes that manage the basic behaviour of other classes, such as creating new instances or defining shared methods (e.g. `__init__` or `__setattr__`). In Python the default metaclass is `type`. Since implementing all of its functionalities anew would be quite difficult, we instead inherit from `type` and let `super()` do the rest. Metaclasses are rarely needed, but debugging is one example where they may be useful. For example, we can log creation of instances or changes in attribute values while leaving the actual logic untouched. We do this in the example below by overriding `type`'s method `__call__`  and adding to classes a custom `__setattr__`. Note that we add this to the `dct`, a dictionary common to all instances of a class using this metaclass. In other words, classes can be thought of as instances of their metaclass. This is why `__new__` defines a class itself, not an instance. Logging in `__new__` would only print messages once the `class Account(metaclass=DebugType):` block is executed. For instance creation, we need the method `__call__`.

In [39]:
class DebugType(type):
    logger = logger
    
    def __new__(cls, name, bases, dct):
        dct['__setattr__'] = cls._debug_setattr
        return super().__new__(cls, name, bases, dct)
        
    def __call__(cls, *args, **kwargs):
        instance = super().__call__(*args, **kwargs)
        cls.logger.debug(f'Instance of class {cls.__name__} created with {args=} and {kwargs=}')
        return instance

    @staticmethod
    def _debug_setattr(instance, name, value):
        cls = instance.__class__
        current_value = None
        try:
            current_value = getattr(instance, name)
        except AttributeError:
            pass
        cls.logger.debug(f'{cls.__name__}: chaning {name} from {current_value} to {value}')
        super(type(instance), instance).__setattr__(name, value)

In this context the strengths of metaclasses are similar to those of wrappers we discussed earlier. They can be very general (class-agnostic), flexible and are easily added, removed or replaced. One difference is that they are, not surprisingly, more class oriented, whereas wrappers could wrap (decorate) any function. We can receive their benefits only with classes that we ourselves defined. For classes defined elsewhere, we may still create a child class using our alternative metaclass, but the tinkering required runs contrary to the aforementioned strengths. Inheriting from built-in classes like `list` can get even more complicated as it requires knowledge of its inner workings. For example, `list` does not use `__setattr__` but rather `__setitem__`, which is behind commands like `L[1] = 2`.

In [40]:
class Account(metaclass=DebugType):
    def __init__(self, name=None, amount=0):
        self.name = name
        self.amount = amount

    def withdraw(self, amount):
        self.amount -= amount

    def transfer(self, other, amount):
        self.withdraw(amount)
        other.withdraw(-amount)

In [41]:
john_acc = Account('John', 230.70)
jane_acc = Account('Jane', 450.00)

00:00:58-DEBUG-at line 21-in func _debug_setattr-Account: chaning name from None to John
00:00:58-DEBUG-at line 21-in func _debug_setattr-Account: chaning amount from None to 230.7
00:00:58-DEBUG-at line 10-in func __call__-Instance of class Account created with args=('John', 230.7) and kwargs={}
00:00:58-DEBUG-at line 21-in func _debug_setattr-Account: chaning name from None to Jane
00:00:58-DEBUG-at line 21-in func _debug_setattr-Account: chaning amount from None to 450.0
00:00:58-DEBUG-at line 10-in func __call__-Instance of class Account created with args=('Jane', 450.0) and kwargs={}


In [42]:
john_acc.transfer(jane_acc, 50)

00:00:58-DEBUG-at line 21-in func _debug_setattr-Account: chaning amount from 230.7 to 180.7
00:00:58-DEBUG-at line 21-in func _debug_setattr-Account: chaning amount from 450.0 to 500.0
